# 使用 Nexus-AM 编译负载

Nexus-AM 是一个裸机运行时测试生成环境。它轻量且易于使用，并且实现了基本的系统调用接口和异常处理程序。此外，它还支持多种 ISA 和配置。

`am/` 下是 nexus-am 框架的源码，`apps/` 和 `tests/` 则是常用的软件负载源码，您也可以创建自己的 app 和 test。

In [4]:
%%bash
cd ../ && source env.sh >/dev/null
cd $AM_HOME

tree -d -L 1

echo apps: $(ls ./apps)
echo tests: $(ls ./tests)

.
├── am
├── apps
├── docs
├─��� libs
├── share
├── tests
└── tools

7 directories
apps: coremark dhrystone dummy-bootrom fceux hello hpmdriver icache-ecc-test litenes litenes_opt loader maprobe mem_test microbench rva-trigger rvv-trigger slider stream tsvc typing zcb-test
tests: aliasgenerator aliastest amtest bitmanip cacheoptest cachetest countertest cputest crypto dualcoretest frequencytest frontendtest klibtest memscantest mmiotest oraclebptest softmdutest softprefetchtest zacas


我们以一开始运行的 Hello, XiangShan 为例，其源码位于 `apps/hello` 目录下，我们将其输出的 “Hello, XiangShan” 替换为 “Welcome to XiangShan Turtorial” 进行编译。

此处传入 ARCH=riscv64-xs 编译到 riscv64 架构的 XiangShan 目标。

默认情况下，riscv64 架构使用 riscv64-unknown-elf- 工具链，本次 tutorial 我们使用 GNU 工具链（riscv64-linux-gnu-），故传入 LINUX_GNU_TOOLCHAIN=1。

参考文件：
- `am/arch/isa/riscv64.mk`
- `am/arch/riscv64-xs.mk`

In [5]:
%%bash
cd ../ && source env.sh >/dev/null
cd $AM_HOME/apps/hello

# 使用 sed 将 “Hello, XiangShan” 替换为 “Welcome to XiangShan Turtorial”
sed -i 's/Hello, XiangShan/Welcome to XiangShan Turtorial/' hello.c

# 进行编译
make ARCH=riscv64-xs LINUX_GNU_TOOLCHAIN=1

# 查看编译输出
ls -l build

# Building hello [riscv64-xs] with AM_HOME {/home/guest/xs-env/nexus-am}
+ CC hello.c
# Building lib-am [riscv64-xs]
+ CC src/noop/isa/riscv/trm.c
+ AS src/nemu/common/mainargs.S
+ CC src/noop/isa/riscv/perf.c
+ CC src/noop/common/uartlite.c
+ CC src/noop/common/serial-16550.c
+ CC src/nemu/isa/riscv/cte.c
+ AS src/nemu/isa/riscv/trap.S
+ CC src/nemu/isa/riscv/cte64.c
+ AS src/nemu/isa/riscv/mtime.S
+ CC src/nemu/isa/riscv/vme.c
+ CC src/xs/common/ioe.c
+ CC src/noop/common/input.c
+ CC src/noop/common/timer.c
+ CC src/noop/isa/riscv/instr.c
+ CC src/xs/isa/riscv/mpe.c
+ CC src/xs/isa/riscv/clint.c
+ CC src/xs/isa/riscv/pmp.c
+ CC src/xs/isa/riscv/plic.c
+ CC src/xs/isa/riscv/pma.c
+ CC src/xs/isa/riscv/cache.c
+ AS src/nemu/isa/riscv/boot/start.S
+ AR -> build/am-riscv64-xs.a
# Building lib-klib [riscv64-xs]
+ CC src/atomic.c
+ CC src/cpp.c
+ CC src/int64.c
+ CC src/io.c
+ CC src/printf.c
+ CC src/stdlib.c
+ CC src/string.c
+ AR -> build/klib-riscv64-xs.a
# Creating binary image [risc

编译得到以下三个文件：
- hello-riscv64-xs.bin：程序二进制镜像（ELF 文件剥离 ELF header 等元数据），供 emu 运行使用
- hello-riscv64-xs.elf：程序的 ELF 文件
- hello-riscv64-xs.txt：程序的反汇编结果，供调试时查看

In [7]:
%%bash
cd ../ && source env.sh >/dev/null
cd $NOOP_HOME

# 使用 emu 运行新编译的软件负载
./build/emu -i $AM_HOME/apps/hello/build/hello-riscv64-xs.bin --no-diff 2>/dev/null

emu compiled at Jul  9 2025, 13:02:33
Using simulated 32768B flash
Core  0's Commit SHA is: 6933700098, dirty: 0
Using simulated 8386560MB RAM
The image is /home/guest/xs-env/nexus-am/apps/hello/build/hello-riscv64-xs.bin
Welcome to XiangShan Turtorial!
Core 0: HIT GOOD TRAP at pc = 0x8000014c
Core-0 instrCnt = 864, cycleCnt = 5840, IPC = 0.147945
Seed=0 Guest cycle spent: 5845 (this will be different from cycleCnt if emu loads a snapshot)
Host time spent: 756ms


当传入的 ARCH 不支持时，make 会打印出所有支持的 ARCH，`|| true` 是为了避免返回值非 0 导致 notebook 报错，您在实际使用中只需要执行 `make ARCH=` 即可

In [8]:
%%bash
cd ../ && source env.sh >/dev/null
cd $AM_HOME

make ARCH= || true

Makefile.check:16: *** Invalid ARCH. Supported: am_native-navy mips32-navy mips32-nemu mips32-sdi native riscv32-navy riscv32-nemu riscv32-noop riscv32-sdi riscv64-navy riscv64-nemu riscv64-noop riscv64-nutshell riscv64-sdi riscv64-xs-dual riscv64-xs-flash riscv64-xs-southlake-flash riscv64-xs-southlake riscv64-xs x86-navy x86-nemu x86-qemu x86-sdi x86_64-qemu.  Stop.
